# Setup infrastructure using docker compose

In [1]:
from jinja2 import Environment, Template
import os

In [2]:
# set env vars

# set correct path here 
base_data_path = "/tmp"

server_port = 5555
test_data_path = base_data_path + "/test_data"
shared_data_path = base_data_path + "/shared_data"
value_key = "POWER_PRODUCTION"

In [3]:
site_ids = [
    "0106fa6a-06d7-4991-a254-93d4ee1464a3",
    "1df9e630-c947-4b9a-89e2-c77b643506c5",
    "2bbfa73e-2a2e-473b-8d1c-f6b87bff8403",
    "50eb6cac-7dbb-44ae-83c6-9b4879a57425",
    "6045bd8d-1754-4963-b3cb-c703110ac5ac",
    "60f50416-f2b0-4191-b82d-c71ba439d558",
    "70118ad9-4de9-473e-8415-27e5f641a7a1",
    "98e065ed-de0e-4973-8e43-7020defe23b0",
    "a62e58d7-8b9a-4531-9ca5-323d136dcec0",
    "a7b10e3f-e8a6-4f58-affd-c50b5f2c4201",
    "aa13d3f3-04b2-43d1-b9ba-16adf87a4bc5",
    "ba7b379a-829d-41cc-bc5d-04eeef3c5020",
    "c8bf65f6-9940-4c65-8ff3-eac19451b1e3",
    "e95b797d-3287-4eaa-b80a-771e840e2d83",
    "ef5b7b47-37d4-4011-8b9f-fffd3020b0d9",
    "018daa06-e6f7-4551-b32c-1a41fc8e0781",
    "13d3ac14-1ba1-4271-bf0f-39e219b97aca",
    "1530d0df-759c-482f-9450-c5e079fbc7ef",
    "189043c3-35bd-43f9-9ca7-0136c10ea23d",
    "19ee674c-fda3-45ee-90b2-8a4c8711459b",
    "3024016a-4586-4f6a-b855-69ef7b91b140",
    "30affaa5-128f-4068-95fb-efd40ea860b5",
    "5767ef9c-25e9-47d2-8249-e044a29e0458",
    "88ff6cd9-f81f-42e5-ad6c-b0ce7fc217bb",
    "8b806cdc-8360-425d-a70d-d2d666097d3e",
    "947c1748-df07-4ff8-845b-073e7ac8eeb9",
    "a2d30421-5731-435a-b7ba-03d37ff0e0ce",
    "ad5085d3-c72a-4a63-bf4f-56cb767b4633",
    "b8c3d9bf-c225-44d3-a64f-11adb3aa0351",
    "d34f66ce-a330-44f1-a8a6-b04e030ffff1"
]

In [5]:
# docker-compose.yml

# Define Docker Compose template as a string
docker_compose_server_template = """
version: '3'
services:
  federated-server:
    container_name: federated-server
    build:
      context: app
      dockerfile: docker/server/Dockerfile
    environment:
      - SERVER_PORT={{ server_port }}
      - SITE_INFO_PATH=/test_data/site_info.csv
      - DATA_PATH=/data
    ports:
      - "{{ server_port }}:{{ server_port }}"
    volumes:
      - server-data:/data
      - {{ test_data_path }}/site_info.csv:/test_data/site_info.csv:ro
    networks:
      - federated-network

volumes:
  server-data:
    external: false

networks:
  federated-network:
    driver: bridge
    external: false

"""

docker_compose_client_template = """
version: '3'
services:
  {% for site_id in site_ids %}
  federated-client-{{ site_id }}:
    container_name: federated-client-{{ site_id }}
    build:
      context: app
      dockerfile: docker/client/Dockerfile
    environment:
      - SITE_ID={{ site_id }}
      - SERVER_ADDRESS=federated-server
      - SERVER_PORT={{ server_port }}
      - TEST_DATA_PATH=/test_data
      - SHARED_DATA_PATH=/shared_data
      - DATA_PATH=/data
      - VALUE_KEY={{ value_key }}
      - RUN_TRAINING={{ run_training }}
      - RUN_PREDICTION={{ run_prediction }}
    volumes:
      - client-{{ site_id }}-data:/data
      - {{ test_data_path + '/' + site_id }}:/test_data:ro
      - {{ shared_data_path }}:/shared_data
    networks:
      - federated-network
  {% endfor %}

volumes:
  {% for site_id in site_ids %}client-{{ site_id }}-data:
    external: false
  {% endfor %}

networks:
  federated-network:
    driver: bridge
    external: false

"""

In [6]:
# Create a Jinja2 template from the string
template_server = Template(docker_compose_server_template)
template_client = Template(docker_compose_client_template)

# Render the template with variables
output_server = template_server.render(server_port=server_port,
                                test_data_path=test_data_path)

output_client_training = template_client.render(server_port=server_port,
                         test_data_path=test_data_path, 
                         shared_data_path=shared_data_path, 
                         value_key=value_key, 
                         run_training=True,
                         run_prediction=False,
                         site_ids=site_ids)

output_client_prediction = template_client.render(server_port=server_port,
                         test_data_path=test_data_path, 
                         shared_data_path=shared_data_path, 
                         value_key=value_key, 
                         run_training=False,
                         run_prediction=True,
                         site_ids=site_ids)

# Write the output to a new Docker Compose file
with open('../docker-compose.server.yml', 'w') as file:
    file.write(output_server)

with open('../docker-compose.client-prediction.yml', 'w') as file:
    file.write(output_client_prediction)

with open('../docker-compose.client-training.yml', 'w') as file:
    file.write(output_client_training)